## Data Preprocessing ##

In [1]:
import pandas as pd 
import json
import csv

In [2]:
data = pd.read_csv("data.csv")
data.head()

,SessionID,TimeStamp,IP,Website,Product,URL_ID,MAC,Gender,Date of Birth,Source Website,Source TimeStamp,Browser,City,Country,State
0,1331799426,15-03-2012 01:17,99.122.210.248,http://www.acme.com/SH55126545/VD55170364,home&garden,6,{7AAB8415-E803-3C5D-7100-E362D7F67CA7},NaN,NaN,sbcglobal.net,15-02-2012 04:16,Firefox,homestead,usa,fl
1,1331800486,15-03-2012 01:34,69.76.12.213,http://www.acme.com/SH55126545/VD55177927,clothing,8,{8D0E437E-9249-4DDA-BC4F-C1E5409E3A3B},F,14 November 1981,rr.com,15-02-2012 01:07,Firefox,coeur d alene,usa,id
2,1331857433,15-03-2012 17:23,67.240.15.94,http://www.acme.com/SH55126545/VD55166807,computers,5,{E3FEBA62-CABA-11D4-820E-00A0C9E58E2D},M,28 April 1982,rr.com,15-02-2012 19:11,Safari,queensbury,usa,ny
3,1331800073,15-03-2012 01:27,98.234.107.75,http://www.acme.com/SH55126545/VD55179433,shoes,9,{49E0D2EE-1D57-48C5-A27D-7660C78CB55C},M,17 January 1996,comcast.net,15-02-2012 00:32,Safari,sunnyvale,usa,ca
4,1331802578,15-03-2012 02:09,75.85.165.38,http://www.acme.com/SH55126545/VD55179433,shoes,9,{F6F8B460-4204-4C26-A32C-B93826EDCB99},F,10 June 1989,rr.com,15-02-2012 00:52,Safari,san diego,usa,ca


In [3]:
s = "{F6F8B460-4204-4C26-A32C-B93826EDCB99}"
s[1: -1]

'F6F8B460-4204-4C26-A32C-B93826EDCB99'

In [4]:
session = {}
products = {}
browserInfo = {}
locationInfo ={}
for index, row in data.iterrows():
    if(row['MAC'][1:-1] in session.keys()):
        session[row['MAC'][1:-1]].append(row['URL_ID'])
        browserInfo[row['MAC'][1:-1]].append(row['Browser'])
        locationInfo[row['MAC'][1:-1]].append(row['Country'])
    else:
        session[row['MAC'][1:-1]] = [row['URL_ID']]
        browserInfo[row['MAC'][1:-1]] = [row['Browser']]
        locationInfo[row['MAC'][1:-1]] = [row['Country']]
    if(row['URL_ID'] not in products.keys()):
        products[row['URL_ID']] = [row["Product"], row["Website"]]

In [5]:
def save_dict_to_file(dic, file):
    f = open(file,'w')
    f.write(str(dic))
    f.close()
# save_dict_to_file(session, "dict.txt")
print(len(session))
final_dict = {key: session[key] for key in session if len(session[key])!=1}
final_browserInfo = {key: browserInfo[key] for key in session if len(session[key])!=1}
final_locationInfo = {key: locationInfo[key] for key in session if len(session[key])!=1}
print(len(final_dict))
key_to_value_lengths = {k:v for k, v in final_dict.items() if v.count(v[0])==len(v)}
for i in key_to_value_lengths.keys():
    del final_dict[i]
    del final_browserInfo[i]
    del final_locationInfo[i]
print(len(final_dict))
print(len(final_browserInfo))
print(len(final_locationInfo))

12348
10893
2638
2638
2638


In [21]:
sessions = list(final_dict.values())
browsers = list(final_browserInfo.values())
print(len(browsers))
locations = list(final_locationInfo.values())

sessionremoved = list(key_to_value_lengths.values())

import  csv
with open("dict_new.txt","w") as f:
    wr = csv.writer(f)
    wr.writerows(sessions)
    
with open("dict_removed.txt","w") as f:
    wr = csv.writer(f)
    wr.writerows(sessionremoved)
# save_dict_to_file(final_dict, "dict_new.txt")

readd = open("dict_new.txt", "r")
contents = readd.read()
contentsnew = contents.replace(",", " ")
contentsnew = contentsnew.replace("\n\n", " -1 -2\n")
newfile = open("save_dict.txt", "w")
newfile.write(contentsnew)
newfile.close()


2638


## Pattern View Data ##

In [23]:
with open("op4.txt", "r") as f:
    op4Content = f.readlines() 
with open("save_dict.txt", "r") as clicksfile:
    clicksdata = clicksfile.readlines()
print(clicksdata[2070])
with open("op1.txt") as op1:
    op1Content = op1.readlines()
with open("op2.txt") as op2:
    op2Content = op2.readlines()
with open("op3.txt") as op3:
    op3Content = op3.readlines()

27 9 8 8 8 8 9 9 9 9 -1 -2



In [25]:
finalPatterns = []
for sLine in op4Content:
    result = sLine.split("-1")
    patternStr = result[0]
    supportIndex = result[1]
    s = ""
    lastStr = "-1 #SUP: "
    pattern = patternStr.strip().split(" ")
    patternList = []
    if(len(pattern)>1):
        for i in range(0,len(pattern)):
            if pattern[i]!=" ":
                s = pattern[i] if s=="" else s + " " + pattern[i]
                stringToBeFound = s + " " + lastStr
                matching = [x for x in eval("op"+str(i+1)+"Content") if x.startswith(stringToBeFound)]

                ## Width
                if(i==0):
                    support = int(matching[0].split(":")[1])
                else:
                    support = int(matching[0].split(":")[1].split(" #")[0])

                ## height
                averageclicks = 0 
                clickpages = sLine.strip().split(": ")[2].split(" ")
                for k in clickpages:
                    clickstring = clicksdata[int(k)].strip().split(" ")
                    if(i>0 and pattern[i] in clickstring[clickstring.index(pattern[i-1]):]):
                        value = clickstring[clickstring.index(pattern[i-1]):].index(pattern[i])+1
                    else:
                        value = clickstring.index(pattern[i])+1
                    averageclicks = averageclicks+ value
                averageclicks = averageclicks/len(clickpages)
                supportsdict = {"id": pattern[i], "label": products[int(pattern[i])][0], "value": int(averageclicks), "width": support, "backgroundColor":"#898d8f"}
                patternList.append(supportsdict)
        clickpages = sLine.split(": ")[2].split(" ")
        averageclicks = 0
        for k in clickpages:
            clickstring = clicksdata[int(k)].split(" ")
            value = len(clickstring)-clickstring.index(pattern[len(pattern)-1])
            averageclicks = averageclicks+ value
        averageclicks = averageclicks/len(clickpages)
        supportsdict = {"id": -1, "label": "last", "value": int(averageclicks), "width": support, "backgroundColor":"#898d8f"}
        patternList.append(supportsdict)
        finalPatterns.append(patternList)    
# newList = [finalPatterns]
print(finalPatterns)

with open("patterns.json", "w") as patternFile:
    for item in finalPatterns:
        patternFile.write("%s\n" % item)
     

[[{'id': '8', 'label': 'clothing', 'value': 3, 'width': 1965, 'backgroundColor': '#898d8f'}, {'id': '27', 'label': 'handbags', 'value': 7, 'width': 105, 'backgroundColor': '#898d8f'}, {'id': -1, 'label': 'last', 'value': 13, 'width': 105, 'backgroundColor': '#898d8f'}], [{'id': '4', 'label': 'electronics', 'value': 15, 'width': 286, 'backgroundColor': '#898d8f'}, {'id': '9', 'label': 'shoes', 'value': 19, 'width': 63, 'backgroundColor': '#898d8f'}, {'id': -1, 'label': 'last', 'value': 38, 'width': 63, 'backgroundColor': '#898d8f'}], [{'id': '2', 'label': 'movies', 'value': 13, 'width': 245, 'backgroundColor': '#898d8f'}, {'id': '9', 'label': 'shoes', 'value': 12, 'width': 74, 'backgroundColor': '#898d8f'}, {'id': -1, 'label': 'last', 'value': 27, 'width': 74, 'backgroundColor': '#898d8f'}], [{'id': '3', 'label': 'games', 'value': 13, 'width': 109, 'backgroundColor': '#898d8f'}, {'id': '4', 'label': 'electronics', 'value': 10, 'width': 66, 'backgroundColor': '#898d8f'}, {'id': -1, 'labe

In [28]:
# finalList = []
# for s in contents:
#     result = s.split("-1")
#     pattern = result[0]
#     support = result[1]
#     s = ""
#     lastStr = "-1 #SUP: "
#     pattern = pattern.strip().split(" ")
#     patternList = []
#     with open("save_dict.txt", "r") as clicksfile:
#         clicksdata = clicksfile.readlines()
#     for i in range(0,len(pattern)):
#         if pattern[i]!=" ":
#             s = pattern[i] if s=="" else s + " " + pattern[i]
#             with open("op"+str(i+1)+".txt", "r") as file:
#                 superfile = file.readlines()
#             stringToBeFound = s+" "
#             stringToBeFound = stringToBeFound + lastStr
#             matching = [x for x in superfile if x.startswith(stringToBeFound)]
#             averageclicks = 0
#             if i==0:
#                 supports = int(matching[0].split(":")[1])
                 
#             else:
#                 supports = int(matching[0].split("#")[1].split(":")[1])
#                 indexlist = matching[0].split("#")[2]
#                 clickpages = indexlist.split(":")[1].split(" ")[1:]
#                 for k in clickpages:
#                     clickstring = clicksdata[int(k)].replace(" ","")
#                     value = clickstring[clickstring.find(pattern[i-1]):].find(pattern[i])
#                     print("===================================")
#                     averageclicks = averageclicks+ value
#                     print(value)
#                 averageclicks = averageclicks/len(clickpages)
#             supportsdict = {"id": pattern[i], "label": products[int(pattern[i])][0], "value": int(averageclicks), "width": supports, "backgroundColor":"#898d8f"}
#             patternList.append(supportsdict)
#     finalList.append(patternList)

# # for i in finalList:
# #     if(len(i)>1):
# #         for j in range(1,len(i)):
# #             i[j-1]["value"] = i[j]["value"]
# #         i[j]["value"] = 0


# ind2remove = [i for i in range(len(finalList)) if len(finalList[i])==1]
# print(ind2remove)
# for i in sorted(ind2remove, reverse=True): 
#     del finalList[i]
# print(len(finalList))
# print("=================Final json=======================")
# newList = [finalList] 
# with open("patterns.json", "w") as patternFile:
#     for item in newList:
#         patternFile.write("%s\n" % item)

## Sequence View Data ##

In [30]:
with open("op4.txt", "r") as f:
    contents = f.readlines()

In [32]:
## Check if the pattern exists in the sequence
def shouldInclude(pattern, sequence):
    pattern = pattern.strip().split(" ")[:-1]
    for i in pattern:
        if(i in sequence):
            sequence = sequence[sequence.index(i):]
        else:
            return False
    return True

In [36]:
print(len(clicksdata))
print("____________________________________")
print(len(browsers))
print(len(locations))

with open("op4.txt", "r") as file:
    sequenceFile = file.readlines()
with open("save_dict.txt", "r") as clicksfile:
    clicksdata = clicksfile.readlines()
for a in range(len(finalPatterns)):
    s = ""
    sequenceList = []
    for i in finalPatterns[a]:
        s = s + " " + str(i["id"])
    sequences = [x for x in sequenceFile if s.strip() in x][0].split(": ")[2].split(" ")
    sequences = [int(x) for x in sequences]
    patterns = [clicksdata[i] for i in range(len(clicksdata)) if i in sequences]
    browsersReq = [browsers[i] for i in range(len(clicksdata)) if i in sequences]
    locationsReq = [locations[i] for i in range(len(clicksdata)) if i in sequences]
    for i in range(len(patterns)):
        temp = patterns[i][:-7].split(" ")
        if(shouldInclude(s, temp)):
            tempList = []
            for j in temp:
                tempList.append(products[int(j)][1])
            sequenceList.append({"sequences": tempList, "Browser": browsersReq[i][0], "Location": locationsReq[i][0]})
            
#     tempDictNew = {}
#     for item in sequenceList:
#         tempDictNew[len(item)] = sequenceList.count(item)
#     print(tempDictNew)
    sequenceDict = {}
    sequenceDict['urls'] = sequenceList
    with open("sequences"+ str(a) + ".json", "w") as sequenceFile1:
        sequenceFile1.write(json.dumps(sequenceDict))    
    visitorDict = {}
    addedKeys = {}
    seqno = 1
    for i in range(len(sequenceDict['urls'])):
        seq = sequenceDict['urls'][i]["sequences"]
        if("".join(seq) not in addedKeys.keys()):
            tempSequences = [d['sequences'] for d in sequenceDict['urls']]
            visitorDict["sequence" + str(seqno)] = [tempSequences.count(seq), sequenceDict["urls"][i]["Browser"], sequenceDict["urls"][i]["Location"]] 
            addedKeys["".join(seq)] = seqno
            seqno = seqno + 1
#         else:
#             visitorDict["sequence" + str(addedKeys["".join(seq)])] = visitorDict["sequence" + str(addedKeys["".join(seq)])]+1
    ## Creating visitor count file
    with open('vistorCount'+str(a)+".csv", 'w') as f: 
        f.write("%s,%s,%s,%s\n"%("sequence","count", "Browser", "Location"))
        for key in visitorDict.keys():
            f.write("%s,%s,%s,%s\n"%(key,visitorDict[key][0],visitorDict[key][1], visitorDict[key][2] ))

print(len(clicksdata))
print("____________________________________")
print(len(browsers))

2638
____________________________________
2638
2638
2638
____________________________________
2638


In [13]:
a = ['http://www.acme.com/SH55126545/VD55165149', 'http://www.acme.com/SH55126545/VD55149415', 'http://www.acme.com/SH55126545/VD55149415', 'http://www.acme.com/SH55126545/VD55149415', 'http://www.acme.com/SH55126545/VD55149415', 'http://www.acme.com/SH55126545/VD55149415', 'http://www.acme.com/SH55126545/VD55165149', 'http://www.acme.com/SH55126545/VD55165149', 'http://www.acme.com/SH55126545/VD55165149', 'http://www.acme.com/SH55126545/VD55149415', 'http://www.acme.com/SH55126545/VD55165149', 'http://www.acme.com/SH55126545/VD55149415', 'http://www.acme.com/SH55126545/VD55165149', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55165149', 'http://www.acme.com/SH55126545/VD55165149', 'http://www.acme.com/SH55126545/VD55165149', 'http://www.acme.com/SH55126545/VD55165149', 'http://www.acme.com/SH55126545/VD55165149', 'http://www.acme.com/SH55126545/VD55165149', 'http://www.acme.com/SH55126545/VD55166807', 'http://www.acme.com/SH55126545/VD55166807', 'http://www.acme.com/SH55126545/VD55166807', 'http://www.acme.com/SH55126545/VD55166807', 'http://www.acme.com/SH55126545/VD55166807', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55149415', 'http://www.acme.com/SH55126545/VD55149415', 'http://www.acme.com/SH55126545/VD55149415']
b = [['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH5580165/VD55173281', 'http://www.acme.com/SH5580165/VD55173281', 'http://www.acme.com/SH5580165/VD55173281', 'http://www.acme.com/SH5580165/VD55173281', 'http://www.acme.com/SH5580165/VD55173281', 'http://www.acme.com/SH5580165/VD55173281'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927'], ['http://www.acme.com/SH55126545/VD55165149', 'http://www.acme.com/SH55126545/VD55165149', 'http://www.acme.com/SH55126545/VD55165149', 'http://www.acme.com/SH55126545/VD55165149', 'http://www.acme.com/SH55126545/VD55165149', 'http://www.acme.com/SH55126545/VD55165149', 'http://www.acme.com/SH55126545/VD55165149', 'http://www.acme.com/SH55126545/VD55165149', 'http://www.acme.com/SH55126545/VD55165149', 'http://www.acme.com/SH55126545/VD55166807', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55165149', 'http://www.acme.com/SH55126545/VD55165149', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55165149', 'http://www.acme.com/SH55126545/VD55165149', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH5580165/VD55173281', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH5580165/VD55173281', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948'], ['http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948'], ['http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433'], ['http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433'], ['http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55179433'], ['http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55179433'], ['http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55179433'], ['http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927'], ['http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948'], ['http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927'], ['http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55173061'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948'], ['http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948'], ['http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948'], ['http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948'], ['http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948'], ['http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927'], ['http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061'], ['http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433'], ['http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55166807', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55166807', 'http://www.acme.com/SH55126545/VD55165149', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55166807', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55166807', 'http://www.acme.com/SH55126545/VD55166807', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55166807', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55166807', 'http://www.acme.com/SH55126545/VD55166807', 'http://www.acme.com/SH55126545/VD55166807', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55166807', 'http://www.acme.com/SH55126545/VD55166807', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927'], ['http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55166807', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55166807', 'http://www.acme.com/SH55126545/VD55166807', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55166807', 'http://www.acme.com/SH55126545/VD55166807', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927'], ['http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55149415', 'http://www.acme.com/SH55126545/VD55149415', 'http://www.acme.com/SH55126545/VD55149415', 'http://www.acme.com/SH55126545/VD55149415', 'http://www.acme.com/SH55126545/VD55149415', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433'], ['http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH5580165/VD55173281', 'http://www.acme.com/SH5580165/VD55173281', 'http://www.acme.com/SH5580165/VD55173281', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55173061'], ['http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55170364', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55179433'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948'], ['http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948'], ['http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433'], ['http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927'], ['http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433'], ['http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433'], ['http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55179433'], ['http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927'], ['http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927'], ['http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927'], ['http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927'], ['http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927'], ['http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948'], ['http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927'], ['http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927'], ['http://www.acme.com/SH5580165/VD55156528', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH5580165/VD55156528'], ['http://www.acme.com/SH5580165/VD55156528', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH5580165/VD55156528'], ['http://www.acme.com/SH5580165/VD55156528', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH5580165/VD55156528'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948'], ['http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927'], ['http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948'], ['http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927'], ['http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH55126545/VD55179433'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927'], ['http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH559040/VD55175948'], ['http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH559040/VD55175948'], ['http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH559040/VD55175948'], ['http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55179433', 'http://www.acme.com/SH559040/VD55175948'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948'], ['http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55166807', 'http://www.acme.com/SH55126545/VD55149415', 'http://www.acme.com/SH55126545/VD55149415', 'http://www.acme.com/SH55126545/VD55149415', 'http://www.acme.com/SH55126545/VD55149415', 'http://www.acme.com/SH55126545/VD55149415', 'http://www.acme.com/SH559040/VD55175948'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH55126545/VD55177927'], ['http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55173061', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948'], ['http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH55126545/VD55177927', 'http://www.acme.com/SH559040/VD55175948', 'http://www.acme.com/SH559040/VD55175948']]
for i in b:
    if(i == a):
        print(True)

In [14]:
a = [[1,3,2], [1,2], [1,2,3]]
a.count([1,2,3])

1